In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import FeatureUnion
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import f1_score
from scipy import stats

In [2]:
from plotly.offline import init_notebook_mode
import plotly.offline as py
import plotly.graph_objs as go
import plotly.express as px
init_notebook_mode(connected=True)

In [3]:
train = "../data/temp/train_2000.csv"
dev = "../data/temp/dev_2000.csv"

##### Load train and dev dataset

In [4]:
train_df = pd.read_csv(train)
dev_df = pd.read_csv(dev)

In [5]:
train_df

,id,same,text1,text2,author1,author2
0,cc9eaace-b00d-57fa-ac0d-5ffb72bfbdf6,False,The shop was a multilevel one. The work area w...,"""In other words the whole thing was a trap, af...",64258,3481055
1,791250bc-c697-587b-bc03-58cc38aef564,False,"Avoiding any pleasantries, she decided to try ...","With a clarity that surprised her, Alice reali...",4126569,10538973
2,3af3ff79-582a-50ba-8380-1e1877e4f5e6,True,"The Princess was now alone in her large room, ...",The team immediately set out a plan to lure Go...,1680860,1680860
3,5ef9b629-100b-5aee-b40e-ed1dd50d6257,True,"""Rizzo?"" she moved her position so that Rizzo ...","""Yes...I like to watch the birds go by,"" she s...",4085372,4085372
4,9450c779-087e-5aae-aac9-18da5f67d7a1,False,"Identifying herself only as Anastasia, she was...","Harry: ""That""s funny when I play that with my ...",5672503,6696853
...,...,...,...,...,...,...
1389,0c1efc81-9417-5afc-8775-1029d4dabdf7,True,Nyx had invited her sons (3) so as not to seem...,"""Come on Princess,"" he teased, tweaking her no...",38709,38709
1390,b47e84fb-053d-5081-80a5-fc9057acc9e5,True,"""Trick? Another one? I thought we were only do...","Oh, how she missed the way he would speak her ...",1126896,1126896
1391,262cef40-f694-5c6f-8f26-cae8177f89ef,False,Hayle was having a hard time to hold her tears...,"Anyways, just so you knows if you ever end up ...",4372830,10381692
1392,0ee63acf-5a66-56c8-ad5a-65aef01c657d,True,"""Captain Tony!"" The female captain scolded, ""D...","brother""s fault! His obsession of Yugioh made ...",909661,909661


In [6]:
texts = np.hstack([train_df['text1'].values, train_df['text2'].values])
texts1 = train_df['text1']
texts2 = train_df['text2']
Y_train = train_df['same'].values
train_df = None

In [7]:
texts

array(['The shop was a multilevel one. The work area was the bottom floor and Twitty guessed someone lived upstairs. He felt kinda uncomfortable in the area. Surfer guys don"t exactly fit in with rough necks. And these guys looked about as rough as they get. One guy looked at him as he wiped his hands from his work. He nodded his head at Twitty, In what our young friend figured was the want to know what he wanted. " Hi. Is Randy James here?" " Yeah. On the roof. Ladder"s outside," " Okay. Thanks man," He followed the instructions that were giving to him and went to find the fire escape ladder . There it was just on the right side, down and everything. He quickly scaled it and found her sitting there blowing smoke from her mouth. " Randy?" " Oh, you found me huh?" She said smiling. " What are you doing up here?" " Thinking. I used to always want to do this when I was younger, but I was to little when we lived out here and when we moved to Miami, we only had one floor and no way to the r

In [8]:
Y_train

array([False, False,  True, ..., False,  True, False])

In [9]:
vectorizer = TfidfVectorizer(analyzer='char_wb', min_df=0.1, vocabulary=None, norm='l1', ngram_range=(1, 6))

In [14]:
x = vectorizer.fit_transform(texts)
scaler = StandardScaler(with_mean=False)
scaler.fit_transform(x)

<2788x29096 sparse matrix of type '<class 'numpy.float64'>'
	with 29397667 stored elements in Compressed Sparse Row format>

In [15]:
x1 = scaler.transform(vectorizer.transform(texts1))
x2 = scaler.transform(vectorizer.transform(texts2))
X_train = np.abs(x1-x2).todense()
texts1=None
texts2=None

## Logistic Regression

In [16]:
clf = LogisticRegression(solver='lbfgs', max_iter=500)
distributions = dict(C=uniform(loc=0, scale=4), penalty=['l2', 'l1'])
param_clf = RandomizedSearchCV(clf, distributions, random_state=0, verbose=2, scoring='roc_auc')
search = param_clf.fit(X_train, Y_train)
search.best_params_

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] C=2.195254015709299, penalty=l1 .................................
[CV] .................. C=2.195254015709299, penalty=l1, total=   0.2s


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\Users\Antonio\miniconda3\envs\authorship\lib\site-packages\sklearn\model_selection\_validation.py:548: FitFailedWarning:

Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Antonio\miniconda3\envs\authorship\lib\site-packages\sklearn\model_selection\_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Antonio\miniconda3\envs\authorship\lib\site-packages\sklearn\linear_model\_logistic.py", line 1304, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "C:\Users\Antonio\miniconda3\envs\authorship\lib\site-packages\sklearn\linear_model\_logistic.py", line 442, in _check_solver
    raise ValueError("Solver %s supports only 'l2' or 'none' penalties, "
ValueError: Solver lbfgs supports only 'l2' or 'none' penaltie

[CV] C=2.195254015709299, penalty=l1 .................................
[CV] .................. C=2.195254015709299, penalty=l1, total=   0.1s
[CV] C=2.195254015709299, penalty=l1 .................................
[CV] .................. C=2.195254015709299, penalty=l1, total=   0.1s
[CV] C=2.195254015709299, penalty=l1 .................................
[CV] .................. C=2.195254015709299, penalty=l1, total=   0.1s
[CV] C=2.195254015709299, penalty=l1 .................................
[CV] .................. C=2.195254015709299, penalty=l1, total=   0.1s
[CV] C=3.3770629943240693, penalty=l1 ................................
[CV] ................. C=3.3770629943240693, penalty=l1, total=   0.1s
[CV] C=3.3770629943240693, penalty=l1 ................................
[CV] ................. C=3.3770629943240693, penalty=l1, total=   0.1s
[CV] C=3.3770629943240693, penalty=l1 ................................
[CV] ................. C=3.3770629943240693, penalty=l1, total=   0.1s
[CV] C

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
C:\Users\Antonio\miniconda3\envs\authorship\lib\site-packages\sklearn\model_selection\_validation.py:548: FitFailedWarning:

Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Antonio\miniconda3\envs\authorship\lib\site-packages\sklearn\model_selection\_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Antonio\miniconda3\envs\authorship\lib\site-packages\sklearn\linear_model\_logistic.py", line 1304, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "C:\Users\Antonio\miniconda3\envs\authorship\lib\site-packages\sklearn\linear_model\_logistic.py", line 442, in _check_solver
    raise ValueError("Solver %s supports only 'l2' or 'none' penalties, "
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties

[CV] C=2.2721782443757292, penalty=l1 ................................
[CV] ................. C=2.2721782443757292, penalty=l1, total=   0.1s
[CV] C=2.2721782443757292, penalty=l1 ................................
[CV] ................. C=2.2721782443757292, penalty=l1, total=   0.1s
[CV] C=2.2721782443757292, penalty=l1 ................................
[CV] ................. C=2.2721782443757292, penalty=l1, total=   0.1s
[CV] C=2.2721782443757292, penalty=l1 ................................
[CV] ................. C=2.2721782443757292, penalty=l1, total=   0.1s
[CV] C=3.34431505414951, penalty=l2 ..................................
[CV] ................... C=3.34431505414951, penalty=l2, total=   7.4s
[CV] C=3.34431505414951, penalty=l2 ..................................
[CV] ................... C=3.34431505414951, penalty=l2, total=   6.3s
[CV] C=3.34431505414951, penalty=l2 ..................................
[CV] ................... C=3.34431505414951, penalty=l2, total=   6.4s
[CV] C

[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:  3.7min finished


{'C': 0.22685190926977272, 'penalty': 'l2'}

In [17]:
clf = LogisticRegression(C=0.226, solver='lbfgs', max_iter=5000, verbose=True)
clf.fit(X_train, Y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    7.2s finished


LogisticRegression(C=0.226, max_iter=5000, verbose=True)

#### test on development data

In [18]:
texts = np.hstack([dev_df['text1'].values, dev_df['text2'].values])
texts1 = dev_df['text1']
texts2 = dev_df['text2']
Y_dev = dev_df['same'].values

In [19]:
x1 = scaler.transform(vectorizer.transform(texts1))
x2 = scaler.transform(vectorizer.transform(texts2))
X_dev = np.abs(x1-x2).todense()

In [20]:
preds = clf.predict_proba(X_dev)[:, 1]
print('FPR-TRP Curve')

fpr, tpr, thresh = roc_curve(Y_dev, preds)
roc_auc = auc(fpr, tpr)
fig = go.Figure()
fig.add_trace(go.Scatter(
    x = fpr,
    y = tpr,
    text = thresh,
    mode='lines'
))
fig.show()
print(roc_auc)


print('P-R Curve')
precision, recall, thresholds = precision_recall_curve(Y_dev, preds)
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=recall,
    y=precision,
    text=np.array(thresholds).astype(str)
))
fig.show()
print('AUC: ', auc(recall, precision))

FPR-TRP Curve


0.920672254533014
P-R Curve


AUC:  0.9386517891942469


In [21]:
preds = clf.predict(X_dev)
f1_score(Y_dev, preds, average='macro')

0.852901902554187

In [22]:
precision, recall, thresholds

(array([0.56227758, 0.56071429, 0.56272401, 0.5647482 , 0.566787  ,
        0.56884058, 0.57090909, 0.5729927 , 0.57509158, 0.57720588,
        0.57933579, 0.58148148, 0.58364312, 0.5858209 , 0.58426966,
        0.58646617, 0.58867925, 0.59090909, 0.59315589, 0.59160305,
        0.59386973, 0.59615385, 0.59459459, 0.59689922, 0.59922179,
        0.6015625 , 0.60392157, 0.60629921, 0.60474308, 0.60714286,
        0.60956175, 0.612     , 0.61445783, 0.61693548, 0.6194332 ,
        0.62195122, 0.62040816, 0.62295082, 0.6255144 , 0.62809917,
        0.63070539, 0.63333333, 0.63598326, 0.63865546, 0.64135021,
        0.6440678 , 0.64680851, 0.64957265, 0.65236052, 0.65517241,
        0.65800866, 0.66086957, 0.66375546, 0.66666667, 0.66960352,
        0.67256637, 0.67111111, 0.67410714, 0.67713004, 0.68018018,
        0.68325792, 0.68636364, 0.68949772, 0.69266055, 0.69585253,
        0.69444444, 0.69767442, 0.70093458, 0.69953052, 0.70283019,
        0.70616114, 0.70952381, 0.70813397, 0.71

### SVM

In [ ]:
clf = SVC(probability = True, random_state = 1)

In [ ]:
param_dist = {"C": stats.uniform(2, 10),
             "gamma": stats.uniform(0.1, 1)}
rand_search = RandomizedSearchCV(clf, param_distributions=param_dist,
                                       n_iter=15,n_jobs=-1)
rand_search.fit(X_train, Y_train)

rand_search.best_params_

In [ ]:
clf = SVC(probability = True, C=4.33, gamma=1.07)
clf.fit(X_train, Y_train)

In [ ]:
texts = np.hstack([dev_df['text1'].values, dev_df['text2'].values])
texts1 = dev_df['text1']
texts2 = dev_df['text2']
Y_dev = dev_df['same'].values

In [ ]:
x1 = scaler.transform(vectorizer.transform(texts1))
x2 = scaler.transform(vectorizer.transform(texts2))
X_dev = np.abs(x1-x2).todense()

In [ ]:
preds = clf.predict_proba(X_dev)[:, 1]
print('FPR-TRP Curve')

fpr, tpr, thresh = roc_curve(Y_dev, preds)
roc_auc = auc(fpr, tpr)
fig = go.Figure()
fig.add_trace(go.Scatter(
    x = fpr,
    y = tpr,
    text = thresh,
    mode='lines'
))
fig.show()
print(roc_auc)


print('P-R Curve')
precision, recall, thresholds = precision_recall_curve(Y_dev, preds)
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=recall,
    y=precision,
    text=np.array(thresholds).astype(str)
))
fig.show()
print('AUC: ', auc(recall, precision))

In [ ]:
preds = (clf.predict(X_dev))
f1_score(Y_dev, preds, average='macro')